In [18]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [19]:
opts = Options()
opts.add_argument("--headless")
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")

service = Service(r"D:\TerrierClub\edgedriver_win64\msedgedriver.exe")
driver = webdriver.Edge(service=service, options=opts)

In [20]:
base_url = "https://terriercentral.bu.edu"
orgs_url = base_url + "/organizations"
print("Opening TerrierCentral...")
driver.get(orgs_url)
time.sleep(5)

# 收集俱乐部链接
links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/organization/']")
org_urls = list({link.get_attribute("href") for link in links if '/organization/' in link.get_attribute("href")})

print(f"Found {len(org_urls)} organization links.")

Opening TerrierCentral...
Found 10 organization links.


In [21]:
data = []
for i, url in enumerate(org_urls[:5]):  # test the first 5 items
    print(f"Processing {i+1}/5: {url}")

    driver.get(url)
    time.sleep(3)

    try:
        title = driver.find_element(By.TAG_NAME, "h1").text.strip()
        intro_elems = driver.find_elements(By.CSS_SELECTOR, "p")
        intro = " ".join([p.text for p in intro_elems if p.text.strip()])
    except Exception as e:
        title, intro = None, None
        print(f"Error at {url}: {e}")

    data.append({"url": url, "title": title, "introduction": intro})
    time.sleep(1)

driver.quit()

Processing 1/5: https://terriercentral.bu.edu/organization/adclub
Processing 2/5: https://terriercentral.bu.edu/organization/accounting-association
Processing 3/5: https://terriercentral.bu.edu/organization/adoptedandmulticultralstudentalliance
Processing 4/5: https://terriercentral.bu.edu/organization/kasualkicks
Processing 5/5: https://terriercentral.bu.edu/organization/asa


In [22]:
df = pd.DataFrame(data)
df.to_csv("bu_clubs.csv", index=False)